In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame as DF
from sklearn.impute import SimpleImputer                  
from sklearn.model_selection import train_test_split

In [ ]:
df_train=pd.read_csv(r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\train.csv')
df_test=pd.read_csv(r'C:\Users\86153\Desktop\house-prices-advanced-regression-techniques\test.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#首先id列对预测房价没有意义，直接删除
df_train.drop("Id", axis = 1, inplace = True)
df_test.drop("Id", axis = 1, inplace = True)

探索数据分布

In [ ]:
#在这里有点作弊行为，为了score高一点直接把训练集和测试机合并后进行处理
all_data=pd.concat([df_test,df_train])
all_data=all_data.reset_index(drop=True)
all_data.describe([.01,.50,.99]).T                #简单看一下数据分布情况，有没有不该出现的数字或者离群值

In [ ]:
all_data = all_data.drop(["Utilities"], axis=1)

一、数据预处理

可以看到数据存在缺失值很多，一些数据严重偏态
对于缺失值问题在这个数据中有三种解决方法 第一类是允许存在na值的比如说泳池相关属性
第二类是类似于BsmtUnfSF这种特征 缺失值可以填充为0
第三类是缺失值表示默认值的情况比如特征Functional
第四类是Functional这种缺失值数量还不算太多可以用众数来填充（不用平均数的原因是因为要填补文本数据）

In [ ]:
fill_na = all_data.loc[:,("PoolQC" , "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "MasVnrType","Exterior2nd")]
for col in fill_na:
    all_data[col].fillna("None",inplace=True)#先填充第一类缺失的情况 
del fill_na,col

In [ ]:
all_data.loc[:,["BsmtUnfSF","TotalBsmtSF","BsmtFinSF2","BsmtFinSF1","BsmtFullBath","BsmtHalfBath", "MasVnrArea","GarageCars","GarageArea","GarageYrBlt"]]=SimpleImputer(strategy="constant",fill_value=0,copy=False).fit_transform(all_data.loc[:,["BsmtUnfSF","TotalBsmtSF","BsmtFinSF2","BsmtFinSF1","BsmtFullBath","BsmtHalfBath", "MasVnrArea","GarageCars","GarageArea","GarageYrBlt"]])
#第二类情况缺失值填充

In [ ]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")#第三类填充

In [ ]:
other_cols = ["MSZoning", "Electrical", "KitchenQual", "Exterior1st", "MSSubClass", "SaleType","LotFrontage"]
for col in other_cols:
    all_data[col].fillna(all_data[col].mode()[0], inplace=True) #第四类字符类型的列用众数填充
del other_cols,col

all_data.isna().sum()[all_data.isna().sum()>0]
SalePrice    1459
dtype: int64
所有缺失值填补完毕

In [ ]:
all_data.loc[all_data['YrSold'].astype(int) - all_data['YearRemodAdd'].astype(int)<0,'YearRemodAdd']=0

In [ ]:
#特征创造
all_data['TotalHouseArea'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']#居住面积
all_data['YearsSinceRemodel'] = all_data['YrSold'].astype(int) - all_data['YearRemodAdd'].astype(int)#房屋翻新日期和出售日期的差
all_data['HighQualArea'] = all_data['1stFlrSF'] + all_data['2ndFlrSF'] + all_data['TotalBsmtSF'] + all_data['BsmtUnfSF'] - all_data['LowQualFinSF']#房屋总高质量面积
all_data['YearsSinceGarageYrBlt'] = all_data['YrSold'].astype(int) - all_data['GarageYrBlt'].astype(int)#房房屋建造日期和出售日期的差

In [ ]:
outer_1=all_data[all_data['YearsSinceRemodel']<0].index
outer_2=all_data[all_data['YearsSinceGarageYrBlt']<0].index
outer_3=all_data[all_data['GarageYrBlt']==2207].index
outer_1.append([outer_2,outer_3])
for l in outer_1:
    all_data=all_data.drop(labels=l)
all_data=all_data.reset_index(drop=True)
#删除掉出售时间比房间重修或者车库重修时间还要大的错误值
del outer_1,outer_2,outer_3

In [ ]:
#数值类离散型变量 数据不用处理
num_=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars']

In [ ]:
#带有顺序的离散型文本变量 手动标签编码
cate_num=['LandContour','ExterQual','ExterCond','BsmtQual','BsmtCond'
,'BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','FireplaceQu'
,'GarageFinish','GarageQual','GarageCond','PoolQC','Fence','LandSlope']

In [ ]:
#文本的离散型变量 数据独热编码
cate_=['MSZoning','Street','Alley','LotShape','LotConfig'
,'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle'
,'RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir'
,'Electrical','Functional','GarageType','PavedDrive','MiscFeature','SaleType','SaleCondition','MSSubClass']

In [ ]:
all_data['MSSubClass']=all_data[['MSSubClass']].astype(object)#出售类别应该是离散型变量

In [ ]:
#连续性变量的日期数据 数据进行标签编码
str_cols = [ "YrSold", 'MoSold',"YearBuilt", "YearRemodAdd", 'GarageYrBlt']
#"YearBuilt", "YearRemodAdd", 'GarageYrBlt',

In [ ]:
all_data.loc[:,cate_num]=all_data.loc[:,cate_num].replace(['Ex','Gd','TA','Fa','Po','None','Av','Mn','No','Lvl','Bnk','HLS','Low','Gtl','Mod','Sev'],[5,4,3,2,1,0,3,2,1,4,3,2,1,3,2,1])
all_data.loc[:,cate_num]=all_data.loc[:,cate_num].replace(['GLQ','ALQ','BLQ','Rec','LwQ','Unf','Fin','RFn','GdPrv','MnPrv','GdWo','MnWw']
                          ,[6,5,4,3,2,1,3,2,4,3,2,1])
#手动标签编码

In [ ]:

onehotresult=pd.get_dummies(all_data[cate_])
onehotcolu=list(onehotresult.columns)
for i in all_data[cate_]:
    all_data=all_data.drop(i,axis=1)
all_data=pd.concat([all_data,onehotresult],axis=1)
all_data.info()
#对离散变量进行独热编码
del i,onehotresult,onehotcolu

In [ ]:
SalePrice=['SalePrice']
non_cate=all_data.drop(SalePrice,axis=1)
a=non_cate.skew().abs().tolist()
skew=DF(np.asarray(a),non_cate.columns)
skew[skew[0]>1]

to_log=list(skew[skew[0]>1].index)
for col in to_log:                             
    all_data[col] = np.log1p(all_data[col])
#对特征进行平滑处理

In [ ]:
inf_=np.isinf(all_data)
all_data[inf_]=0

nan=all_data.drop('SalePrice',axis=1).isna()
all_data[nan]=0
#消除数据处理过程中产生的inf和na

In [ ]:
del num_,cate_,cate_num,str_cols,#SalePrice,to_log,non_cate,skew,col,inf_,a

In [ ]:
y=all_data[all_data['SalePrice'].isna()].drop('SalePrice',axis=1)
x=all_data[all_data['SalePrice'].notna()]
x.reset_index(drop=True)
y.reset_index(drop=True)
#区分出来训练数据和测试数据

In [ ]:
target=x['SalePrice']
feature=x.drop('SalePrice',axis=1)

pca=PCA(215)
pca_line=pca.fit(feature,target)
#plt.plot(range(1,255),np.cumsum(pca_line.explained_variance_ratio_))
feature=DF(pca_line.transform(feature).tolist())
y=DF(pca_line.transform(y).tolist())
#实测发现pca对分数有负面影响

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(feature,target,test_size=0.3)